In [1]:
import pandas as pd
import numpy as np
name = "billing_and_accruals_for_script_2025-07-01_07-57-10"

In [2]:
dtype_dict = {
    'КПП': str,
}
dataset = pd.read_excel('/home/boris/Downloads/' + name + '.xlsx', dtype=dtype_dict)
dataset['КПП'] = dataset['КПП'].astype(str)
dataset['Начисления partner'] = dataset['Начисления partner'].fillna(0)
dataset['Сумма-test'] = dataset['Сумма']

In [3]:
dataset = dataset.sort_values(by=['Название компании', 'Дата', 'Название номенклатуры '])
dataset

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner,Тариф,service,Сумма-test
14526,1,2025-01-31,155522.0,Подписка ПРО ЛК 155522,1,ФФФФФФФФФ,Подписка ПРО ЛК 155522,40.0,Подписка ПРО,2.7,0.0,0.0,NaN,pro,0.0
14527,1,2025-02-28,155522.0,Подписка ПРО ЛК 155522,1,ФФФФФФФФФ,Подписка ПРО ЛК 155522,40.0,Подписка ПРО,2.7,0.0,0.0,PRO без биллинга,pro,0.0
14528,1,2025-02-28,155522.0,NaN,1,ФФФФФФФФФ,NaN,NaN,NaN,0.0,0.0,0.0,PRO без биллинга,pro_free,0.0
14529,1,2025-03-31,155522.0,NaN,1,ФФФФФФФФФ,NaN,NaN,NaN,0.0,0.0,0.0,PRO без биллинга,pro_free,0.0
14530,1,2025-04-30,155522.0,NaN,1,ФФФФФФФФФ,NaN,NaN,NaN,0.0,0.0,0.0,Start,pro_free,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11071,NaN,2025-07-31,131611.0,131611,0326483040,032601001,131611,10.0,Архивация видеопотока с заданной глубиной хран...,4.0,4.0,0.0,NaN,archive_3,4.0
13384,NaN,2025-07-31,144050.0,144050,910200020767,nan,144050,50.0,Архивация видеопотока с заданной глубиной хран...,2.8,5.6,0.0,NaN,archive_3_pro,5.6
8314,NaN,2025-07-31,123786.0,Подписка ПРО ЛК 123786,7627029851,434501001,Подписка ПРО ЛК 123786,40.0,Подписка ПРО,2.7,486.0,0.0,NaN,pro,486.0
13383,NaN,2025-07-31,144050.0,Подписка ПРО ЛК 144050,910200020767,nan,Подписка ПРО ЛК 144050,40.0,Подписка ПРО,2.7,337.5,0.0,NaN,pro,337.5


In [4]:
for i in range(0,dataset.shape[0]):
    if i == 0:
        date = dataset.loc[0,'Дата']
        lk = dataset.loc[0,'Личный кабинет'] 
        dataset.loc[0,'ballance'] = 0
        dataset.loc[0,'ballance'] = dataset.loc[0,'Сумма'] - dataset.loc[0, 'Начисления partner']
        if dataset.loc[0,'ballance'] < 0:
            dataset.loc[0,'Сумма']  = 0
        if dataset.loc[0,'ballance'] >= 0:
                dataset.loc[0,'Сумма'] = dataset.loc[0,'ballance'] 
                dataset.loc[0,'ballance'] = 0
            
    else:
    
        if date == dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if date != dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] - dataset.loc[i, 'Начисления partner'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if lk != dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            lk = dataset.loc[i,'Личный кабинет']
            
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] \
                                        - dataset.loc[i, 'Начисления partner']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0



In [5]:
dataset.to_excel('/home/boris/Downloads/'+ name + '_test.xlsx', index=False)

In [6]:
dataset_ready = dataset.drop(['Начисления partner','Сумма-test','ballance'], axis= 1)

In [7]:
dataset_ready['Количество'] = round(dataset_ready['Сумма']/dataset_ready['Стоимость'])
dataset_ready['Количество'] = dataset_ready['Количество'].fillna(0) 

In [8]:
dataset_ready.to_excel('/home/boris/Downloads/'+ name + '_ready.xlsx', index=False)